# 5. トレーニング実行


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードの部分について、ソクラテス式チャットボットの学習を例に説明させていただきます。

### 1. チェックポイント管理 (5.1の前半部分)

チェックポイントとは、モデルの学習途中の状態を保存したものです。例えば：

- ソクラテスボットが100個の対話を学習した時点
- 200個の対話を学習した時点
- 300個の対話を学習した時点

というように、定期的に学習状態を保存します。これは以下のような理由で重要です：

1. **学習の中断と再開**：
   - 例：深夜にサーバーがメンテナンスで停止しても、最後のチェックポイントから再開できる
   - 例：学習中にエラーが発生しても、最後の安定した状態から再開できる

2. **進捗確認**：
   - どれだけの対話を学習したか
   - ソクラテス式の問答がどの程度上手くなってきているか
   を確認できます

コードでは、チェックポイントのディレクトリを確認し、以前の学習データがあれば、その状態を確認します。もし学習が既に完了していれば（設定したエポック数に達していれば）、誤って再学習することを防ぐために処理を終了します。

### 2. モデルの学習実行 (5.1の後半部分)

`trainer.train()`で実際の学習を開始します。この過程で：
- ソクラテス式の対話データを少しずつモデルに学習させる
- 定期的に学習の進捗を確認する
- 必要に応じてチェックポイントを保存する

といった処理が行われます。

### 3. ベストモデルと設定の保存 (5.2)

学習が完了したら、以下の情報を保存します：

1. **学習設定の保存**：
   - どのような設定で学習を行ったか（学習率、バッチサイズなど）
   - どのようなモデルを使用したか
   などの情報を`training_config.json`として保存

2. **最良のモデルの保存**：
   - 学習中で最も性能の良かったモデル（最も上手くソクラテス式の対話ができるようになった状態）を保存
   - このモデルの性能指標（どれくらい上手く対話ができるか）も一緒に保存

3. **トークナイザーの保存**：
   - モデルが文章を理解するために必要な単語分割の規則も保存

これらの情報は、後で：
- モデルを使って実際にチャットボットを動かす時
- 学習設定を改善する時
- 追加の学習を行う時
などに必要となります。

このように、このセクションは「学習の実行」「進捗管理」「結果の保存」という、モデル学習の重要な工程を管理している部分だと言えます。

        
</div>
    </details>
</div>


In [ ]:

### 5.1 チェックポイント管理とトレーニング実行

# Start training
logging.info("Starting training...")
try:
    checkpoint_dir = MODEL_OUTPUT_DIR  
    resume_from_checkpoint = None
    
    # Check if running in Kaggle environment
    is_kaggle = os.path.exists('/kaggle/working')
    
    # Checkpoint status and processing
    if os.path.exists(checkpoint_dir):
        print("\nChecking checkpoint status...")  
        checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
        if checkpoints:
            # Get latest checkpoint
            latest_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[1]))
            checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
            print(f"Found latest checkpoint: {latest_checkpoint}") 
            
            # Check checkpoint status
            state_path = os.path.join(checkpoint_path, "trainer_state.json")
            if os.path.exists(state_path):
                with open(state_path, 'r') as f:
                    state = json.load(f)
                current_epoch = state.get('epoch', 0)
                print(f"\nCurrent training status:")  
                print(f"Current epoch: {current_epoch}")  
                print(f"Target epochs: {training_args.num_train_epochs}")  
                
                # Exit safely if completed
                if current_epoch >= training_args.num_train_epochs - 0.1:
                    print("\n" + "="*50)
                    print("IMPORTANT NOTICE:")
                    print(f"Training has already been completed at epoch {current_epoch}!")
                    print(f"Target epochs was {training_args.num_train_epochs}")  
                    print(f"Trained model is available at: {checkpoint_dir}")
                    print("="*50 + "\n")
                    logging.info("Training has already been completed. Exiting to protect existing model.")
                    logging.info(f"Trained model is available at: {checkpoint_dir}")
                    exit(0)
            else:
                logging.warning("Invalid checkpoint state found. Please check manually.")
                logging.warning(f"Checkpoint directory: {checkpoint_dir}")
        else:
            logging.warning("Checkpoint directory exists but no checkpoints found.")
            logging.info("Continuing with training...")  # 追加: 自動的に続行

    # Start training (or resume)
    trainer.train(resume_from_checkpoint=resume_from_checkpoint)
    logging.info("Training completed successfully!")



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードについて、ソクラテス式チャットボットのトレーニングを例に説明させていただきます。

### チェックポイントとは？
チェックポイントとは、モデルの学習途中の状態を保存したものです。例えば、ソクラテス式チャットボットの学習において、「問答形式での対話の仕方」を学習している途中で、何らかの理由で学習が中断された場合に、最初からやり直すのではなく、途中から再開できるようにするための仕組みです。

### コードの主な機能

1. **チェックポイントの確認**
```python
if os.path.exists(checkpoint_dir):
    print("\nChecking checkpoint status...")  
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
```
- まず、以前の学習データ（チェックポイント）が存在するかを確認します
- 例：前回の学習で「ソクラテスのような質問の仕方」を5時間学習した後に中断した場合、その続きから始められるようにします

2. **最新のチェックポイントを見つける**
```python
if checkpoints:
    latest_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[1]))
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
```
- 複数のチェックポイントがある場合、最新のものを探します
- 例：「checkpoint-1000」（1000ステップ目）、「checkpoint-2000」（2000ステップ目）があった場合、2000ステップ目の方を選びます

3. **学習の進捗確認**
```python
current_epoch = state.get('epoch', 0)
print(f"Current epoch: {current_epoch}")  
print(f"Target epochs: {training_args.num_train_epochs}")  
```
- 現在の学習回数（エポック）と目標の学習回数を確認します
- 例：30エポックの学習を予定していて、既に25エポック終わっていれば、残り5エポックだと分かります

4. **学習完了チェック**
```python
if current_epoch >= training_args.num_train_epochs - 0.1:
    print("Training has already been completed!")
    exit(0)
```
- 既に学習が完了している場合は、誤って再学習することを防ぎます
- 例：ソクラテス式の対話の仕方を既に十分学習済みの場合、不要な追加学習を防ぎます

5. **学習の開始または再開**
```python
trainer.train(resume_from_checkpoint=resume_from_checkpoint)
```
- チェックポイントがある場合はそこから再開し、ない場合は新規に学習を開始します
- 例：
  - 新規の場合：ソクラテス式の対話の学習を一から開始
  - 再開の場合：前回の「なぜそう考えるのですか？」といった質問の仕方を学んだところから継続

このシステムにより、長時間かかる学習プロセスを安全に管理し、途中で中断しても再開できる仕組みを実現しています。まるで、本を読むときのしおりのような役割を果たしているのです。

        
</div>
    </details>
</div>


In [ ]:

    ### 5.2 ベストモデルと設定の保存
    # Save settings (as JSON)
    def convert_to_serializable(obj):
        if isinstance(obj, set):
            return list(obj)
        elif isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [convert_to_serializable(x) for x in obj]
        return obj

    # Convert each setting
    training_args_dict = convert_to_serializable(training_args.to_dict())
    lora_config_dict = convert_to_serializable(lora_config.to_dict())

    config_dict = {
        "model_name": model_name,
        "training_args": training_args_dict,
        "lora_config": lora_config_dict,
        "bnb_config": {
            "load_in_4bit": bnb_config.load_in_4bit,
            "bnb_4bit_use_double_quant": bnb_config.bnb_4bit_use_double_quant,
            "bnb_4bit_quant_type": bnb_config.bnb_4bit_quant_type,
            "bnb_4bit_compute_dtype": str(bnb_config.bnb_4bit_compute_dtype),
        }
    }
    
    # Save configurations
    with open(os.path.join(training_args.output_dir, "training_config.json"), "w", encoding="utf-8") as f:
        json.dump(config_dict, f, indent=2, ensure_ascii=False)
    
    # トレーナーが保持している最良のモデルを保存
    # load_best_model_at_end=Trueにより、この時点で既にbestモデルがロードされている
    best_model_path = os.path.join(training_args.output_dir, "best_model")
    os.makedirs(best_model_path, exist_ok=True)
    
    # Save best model and its configuration
    trainer.model.save_pretrained(best_model_path)
    model.config.save_pretrained(best_model_path)
    tokenizer.save_pretrained(best_model_path)
    
    # Save a marker file indicating this is the best model
    with open(os.path.join(best_model_path, "best_model_info.json"), "w", encoding="utf-8") as f:
        best_metrics = {
            "best_metric": trainer.state.best_metric,
            "best_model_checkpoint": trainer.state.best_model_checkpoint,
            "best_perplexity": trainer.state.best_metric
        }
        json.dump(best_metrics, f, indent=2)
    
    logging.info(f"Best model saved to {best_model_path}")
    logging.info(f"Best perplexity: {trainer.state.best_metric}")
    logging.info("Model and configuration saved successfully!")

except Exception as e:
    logging.error(f"An error occurred: {str(e)}")
    raise


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードについて、ソクラテス式チャットボットの例を用いて説明させていただきます。

### 1. データの保存形式の準備
```python
def convert_to_serializable(obj):
    if isinstance(obj, set):
        return list(obj)
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(x) for x in obj]
    return obj
```
これは、複雑なデータ構造をJSON形式で保存できるように変換する関数です。
例：ソクラテス式の対話パターンのセット（集合）を保存可能なリスト形式に変換します。

### 2. 設定情報の保存
```python
config_dict = {
    "model_name": model_name,
    "training_args": training_args_dict,
    "lora_config": lora_config_dict,
    "bnb_config": {
        "load_in_4bit": bnb_config.load_in_4bit,
        ...
    }
}
```
モデルの設定情報をまとめて保存します。例えば：
- モデル名：「google/gemma-2-2b-jpn-it」
- 学習設定：エポック数30回、学習率2e-4など
- LoRA設定：質問の仕方を効率的に学習するための設定
- 量子化設定：モデルを軽量化するための設定

### 3. ベストモデルの保存
```python
best_model_path = os.path.join(training_args.output_dir, "best_model")
trainer.model.save_pretrained(best_model_path)
model.config.save_pretrained(best_model_path)
tokenizer.save_pretrained(best_model_path)
```
最も性能の良かったモデルを保存します。例えば：
- 30エポックの学習の中で、最も「ソクラテス式の対話」が上手くできたモデルを保存
- モデルの設定と、単語を処理するためのトークナイザーも一緒に保存

### 4. ベストモデルの性能情報の保存
```python
best_metrics = {
    "best_metric": trainer.state.best_metric,
    "best_model_checkpoint": trainer.state.best_model_checkpoint,
    "best_perplexity": trainer.state.best_metric
}
```
モデルの性能指標を保存します。例：
- パープレキシティ（文章の自然さを示す指標）
- どのチェックポイントが最も良かったか
- いつの時点で最高の性能を記録したか

### 5. エラー処理
```python
except Exception as e:
    logging.error(f"An error occurred: {str(e)}")
    raise
```
保存処理中に問題が発生した場合（ディスク容量不足など）、エラーログを記録して通知します。

このコードは、まるでソクラテス式チャットボットの「完成品」と「作り方のレシピ」の両方を保存するようなものです。後で同じような対話モデルを作りたい場合や、モデルを改良したい場合に、これらの情報が重要な参考資料となります。

        
</div>
    </details>
</div>
